Importing all the libraries

In [1]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torch
from torchvision import models
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pdB
from sklearn.metrics import roc_auc_score, auc, roc_curve
from torch.utils.data import Dataset
from PIL import Image
import os
from torchvision import datasets, transforms
from shutil import rmtree, copytree, copyfile
import random
import warnings
from google.colab import drive


Model1-inception v3

In [ ]:
class InceptionV3(nn.Module):
  def __init__(self, num_classes = 2):
    super(InceptionV3, self).__init__()
    model = models.inception_v3(pretrained=True, aux_logits=False)
    self.Conv2d_1a_3x3 = model.Conv2d_1a_3x3
    self.Conv2d_2a_3x3 = model.Conv2d_2a_3x3
    self.Conv2d_2b_3x3 = model.Conv2d_2b_3x3
    self.Conv2d_3b_1x1 = model.Conv2d_3b_1x1
    self.Conv2d_4a_3x3 = model.Conv2d_4a_3x3
    self.Mixed_5b = model.Mixed_5b
    self.Mixed_5c = model.Mixed_5c
    self.Mixed_5d = model.Mixed_5d
    self.Mixed_6a = model.Mixed_6a
    self.Mixed_6b = model.Mixed_6b
    self.Mixed_6c = model.Mixed_6c
    self.Mixed_6d = model.Mixed_6d
    self.Mixed_6e = model.Mixed_6e
    self.Mixed_7a = model.Mixed_7a
    self.Mixed_7b = model.Mixed_7b
    self.Mixed_7c = model.Mixed_7c

    self.dropout = nn.Dropout(p=0.5)
    self.fc = nn.Linear(2048, num_classes)
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

  def forward(self, x):
      features = None
      x = nn.Upsample(size=(299, 299), mode='bilinear')(x)
      x = self.Conv2d_1a_3x3(x)
      x = self.Conv2d_2a_3x3(x)
      x = self.Conv2d_2b_3x3(x)
      x = F.max_pool2d(x, kernel_size=3, stride=2)
      x = self.Conv2d_3b_1x1(x)
      x = self.Conv2d_4a_3x3(x)

      x = F.max_pool2d(x, kernel_size=3, stride=2)
      x = self.Mixed_5b(x)
      x = self.Mixed_5c(x)
      x = self.Mixed_5d(x)

      x = self.Mixed_6a(x)
      x = self.Mixed_6b(x)
      x = self.Mixed_6c(x)
      x = self.Mixed_6d(x)
      x = self.Mixed_6e(x)

      # image region features
      features = x

      x = self.Mixed_7a(x)
      x = self.Mixed_7b(x)
      x = self.Mixed_7c(x)
      x = self.avgpool(x)
      x = self.dropout(x)
      x = x.view(x.size(0), -1)

      x = self.fc(x)
      return x 

Model2-ResNet34

In [ ]:
class Resnet34(nn.Module):
  def __init__(self, num_classes = 2):
    super(Resnet34, self).__init__()
    model = models.resnet34(pretrained=True)
    self.conv1 = model.conv1
    self.bn1 = model.bn1
    self.relu = model.relu
    self.maxpool = model.maxpool
    self.layer1 = model.layer1
    self.layer2 = model.layer2
    self.layer3 = model.layer3
    self.layer4 = model.layer4
    self.avgpool = model.avgpool
    self.__in_features = model.fc.in_features
    self.fc = nn.Linear(512, num_classes)
    # self.softmax = nn.Softmax()
  def forward(self, x):

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    # x = self.softmax(x)
    return x

  def output_num(self):
    return self.__in_features

Model3-DenseNet121

In [ ]:
class Densnet121(nn.Module):
  def __init__(self, num_classes = 2):
    super(Densnet121, self).__init__()
    model = models.densenet121(pretrained=True)
    self.features = model.features
    self.fc = nn.Linear(1024, num_classes)
    self.relu_out = 0

  def forward(self, x):
    features = self.features(x)
    out = F.relu(features, inplace=True)
    self.relu_out = out
    out = F.avg_pool2d(out, kernel_size=7, stride=1).view(features.size(0), -1)
    out = self.fc(out)
    return out

  def cam_out(self):
      return self.relu_out

Model4-ResNeXt50

In [ ]:
class ResNeXt50(nn.Module):

    def __init__(self, num_classes=2):
        super(ResNeXt50, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)
        #resnext101_32x4d = torch.hub.load('pytorch/vision:v0.10.0', 'resnext101_32x4d', pretrained=True)
        self.num_classes = num_classes
        #self.conv_1_3x3 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.conv1 = model.conv1
        self.bn1 = model.bn1
        self.relu = model.relu
        self.layer1 = model.layer1
        self.layer2 = model.layer2
        self.layer3 = model.layer3
        #self.features = model.features
        #self.avg_pool = nn.AvgPool2d((7, 7), (1, 1))
        self.avg_pool = model.avgpool
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, input):
        x = self.conv1(input)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

Model 5- Wide ResNet50

In [ ]:
class WideResnet50(nn.Module):
  def __init__(self, num_classes = 2):
    super(WideResnet50, self).__init__()
    model = models.wide_resnet50_2(pretrained=True)
    self.conv1 = model.conv1
    self.maxpool = model.maxpool
    self.layer1 = model.layer1
    self.layer2 = model.layer2
    self.layer3 = model.layer3
    self.avgpool = model.avgpool
    self.fc = nn.Linear(1024, num_classes)
  def forward(self, x):

    x = self.conv1(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

Data Preperation

In [ ]:
def load_training(root_path, dir, batch_size, kwargs):
    transform = transforms.Compose(
        [transforms.Resize([256, 256]),
         transforms.RandomRotation(15),
         transforms.ColorJitter(),
         transforms.RandomCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.RandomVerticalFlip(),
         transforms.ToTensor()
         ])
    data = datasets.ImageFolder(root=os.path.join(root_path, dir), transform=transform)
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True, **kwargs)
    return train_loader

def load_testing(root_path, dir, batch_size, kwargs):

    transform = transforms.Compose(
        [transforms.Resize([224, 224]),
         transforms.ToTensor()])
    data = datasets.ImageFolder(root=os.path.join(root_path, dir), transform=transform)
    # print(list(data.imgs))
    names = list(map(lambda x: os.path.basename(x[0]), list(data.imgs)))
    label = list(map(lambda x: x[1], list(data.imgs)))
    # print(names, label)
    # for name, label in data.imgs:
    #     print(name, label)
    test_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, **kwargs)
    return test_loader, names, label

Training and plotting functions

In [ ]:
import time

def train(model, train_dl, val_dl, epochs=100, device='cpu'):


    history = {} # Collects per-epoch loss and acc like Keras' fit().
    history['loss'] = []
    history['val_loss'] = []
    history['acc'] = []
    history['val_acc'] = []

    start_time_sec = time.time()

    for epoch in range(1, epochs+1):
        LEARNING_RATE = max(lr * (0.1 ** (epochs // 100)), 1e-5)
        optimizer = torch.optim.SGD([{'params': model.parameters()}], lr=LEARNING_RATE, momentum=momentum, weight_decay=l2_decay)

        # --- TRAIN AND EVALUATE ON TRAINING SET -----------------------------
        model.train()
        train_loss         = 0.0
        num_train_correct  = 0
        num_train_examples = 0

        for batch in train_dl:

            optimizer.zero_grad()

            x    = batch[0].to(device)
            y    = batch[1].to(device)
            yhat = model(x)
            optimizer.zero_grad()
            loss = F.nll_loss(F.log_softmax(yhat, dim=1), y)

            loss.backward()
            optimizer.step()

            train_loss         += loss.data.item() * x.size(0)
            num_train_correct  += (torch.max(yhat, 1)[1] == y).sum().item()
            num_train_examples += x.shape[0]

        train_acc   = num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_dl.dataset)


        # --- EVALUATE ON VALIDATION SET -------------------------------------
        model.eval()
        val_loss       = 0.0
        num_val_correct  = 0
        num_val_examples = 0

        for batch in val_dl:

            x    = batch[0].to(device)
            y    = batch[1].to(device)
            yhat = model(x)
            loss=F.nll_loss(F.log_softmax(yhat, dim=1), y, size_average=False)

            val_loss         += loss.data.item() * x.size(0)
            num_val_correct  += (torch.max(yhat, 1)[1] == y).sum().item()
            num_val_examples += y.shape[0]

        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_dl.dataset)


        if epoch == 1 or epoch % 10 == 0:
          print('Epoch %3d/%3d, train loss: %5.2f, train acc: %5.2f, val loss: %5.2f, val acc: %5.2f' % \
                (epoch, epochs, train_loss, train_acc, val_loss, val_acc))

        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['acc'].append(train_acc)
        history['val_acc'].append(val_acc)

    # END OF TRAINING LOOP


    end_time_sec       = time.time()
    total_time_sec     = end_time_sec - start_time_sec
    time_per_epoch_sec = total_time_sec / epochs
    print()
    print('Time total:     %5.2f sec' % (total_time_sec))
    print('Time per epoch: %5.2f sec' % (time_per_epoch_sec))

    return history

def plot_loss(history, model_name):
  plt.plot(history['loss'])
  plt.plot(history['val_loss'])
  plt.title(model_name +' Loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

def plot_accuracy(history, model_name):
  plt.plot(history['acc'])
  plt.plot(history['val_acc'])
  plt.title(model_name +' Accuracy')
  plt.ylabel('acc')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

Fixing the training parameters

In [ ]:
batch_size = 8
epochs = 20
lr = 0.01
momentum = 0.8
no_cuda = False
seed = 8
log_interval = 10
l2_decay = 5e-4
random_seed = 80
split_train_ratio=0.2

path = "drive/MyDrive/Dataset_BUSI_with_GT/"
source_name = "train"
target_name = "val"

Calling GPU support

In [ ]:
cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
kwargs = {'num_workers': 4, 'pin_memory': True} if cuda else {}

Creating Data Loaders

In [ ]:
num_classes = len(os.listdir(os.path.join(path, source_name)))
source_loader = load_training(path, source_name, batch_size, kwargs)
target_test_loader, names, label = load_testing(path, target_name, batch_size, kwargs)

len_source_dataset = len(source_loader.dataset)
len_target_dataset = len(target_test_loader.dataset)
len_source_loader = len(source_loader)

Model 1 training

In [ ]:
model_inc = InceptionV3(num_classes=num_classes)
model_inc.cuda()
history_inc=train(model_inc, source_loader, target_test_loader, epochs=50, device='cuda')

Model 2 training

In [ ]:
model_res=Resnet34(num_classes=num_classes)
model_res.cuda()
history_res=train(model_res, source_loader, target_test_loader, epochs=50, device='cuda')

Model 3 training

In [ ]:
model_dense=Densnet121(num_classes=num_classes)
model_dense.cuda()
history_dense=train(model_dense, source_loader, target_test_loader, epochs=50, device='cuda')

Model 4 training

In [ ]:
#model_next=models.resnext50_32x4d(pretrained=True)
model_next = ResNeXt50()
model_next.cuda()
history_next=train(model_next, source_loader, target_test_loader, epochs=50, device='cuda')

Model 5 training

In [ ]:
model_wres = WideResnet50()
model_wres.cuda()
history_wres=train(model_wres, source_loader, target_test_loader, epochs=50, device='cuda')

Plotting the accuracy and loss curve for each

In [ ]:
plot_accuracy(history_inc, 'InceptionV3')
plot_loss(history_inc, 'InceptionV3')
plot_accuracy(history_res, 'ResNet')
plot_loss(history_res, 'ResNet')
plot_accuracy(history_dense, 'DenseNet')
plot_loss(history_dense, 'DenseNet')
plot_accuracy(history_next, 'ResNeXt')
plot_loss(history_next, 'ResNeXt')
plot_accuracy(history_wres, 'Wide ResNet')
plot_loss(history_wres, 'Wide ResNet')

Code for the ensemble network

In [ ]:
def output(model, data):
    model.eval()
    s_output = model(data)
    M_possibility = float(F.softmax(s_output).data.cpu().numpy()[0][1])
    return M_possibility


def get_result_by_model(image, model):
    result = output(model, image)

    if result >= .5:
        return 1
    else:
        return 0

def get_result_birads(image):
    result = (.2)*output(model_inc, image)  + (.2)*output(model_res, image) + (.2)*output(model_dense, image) + (.2)*output(model_next, image) + (.2)*output(model_wres, image)
    if result >= 0.5:
        return 1
    else:
        return 0

def get_result_tirads(image):
    result = (.2)*output(model_inc, image)  + (.2)*output(model_res, image) + (.2)*output(model_dense, image) + (.2)*output(model_next, image) + (.2)*output(model_wres, image)
    if result >= 0.35:
        return 1
    else:
        return 0

def get_result_birads_v2(image):
    result = (.25)*output(model_inc, image)  + (.25)*output(model_res, image) + (.25)*output(model_next, image) + (.25)*output(model_wres, image)
    if result >= 0.5:
        return 1
    else:
        return 0

def get_pytorch_image(name):
    transform = transforms.Compose(
        [transforms.Resize([224, 224]),
         transforms.ToTensor()])
    image = Image.open(name)
    image = image.convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0)
    image = image.cuda()
    return image

Loading test data

In [ ]:
test_load, name1, label1 = load_testing(path, 'test', batch_size, kwargs)

Performance of ensemble 2

In [ ]:
pred_label=[]
for n,l in zip(name1,label1):
  if l==0:
    img=path+'test/benign/'+n;
  else:
    img=path+'test/malignant/'+n;
  img=get_pytorch_image(img)
  temp=get_result_birads_v2(img)
  pred_label.append(temp)

from sklearn.metrics import accuracy_score,average_precision_score, f1_score,recall_score, precision_score
acc=accuracy_score(label1,pred_label)
f1=f1_score(label1,pred_label)
re=recall_score(label1,pred_label)
pre=precision_score(label1,pred_label)

print(acc)
print(f1)
print(re)
print(pre)

performance of ensemble 1

In [ ]:
pred_label=[]
for n,l in zip(name1,label1):
  if l==0:
    img=path+'test/benign/'+n;
  else:
    img=path+'test/malignant/'+n;
  img=get_pytorch_image(img)
  temp=get_result_birads(img)
  pred_label.append(temp)

from sklearn.metrics import accuracy_score,average_precision_score, f1_score,recall_score, precision_score
acc=accuracy_score(label1,pred_label)
f1=f1_score(label1,pred_label)
re=recall_score(label1,pred_label)
pre=precision_score(label1,pred_label)

print(acc)
print(f1)
print(re)
print(pre)

Performance of model 2

In [ ]:
pred_label=[]
for n,l in zip(name1,label1):
  if l==0:
    img=path+'test/benign/'+n;
  else:
    img=path+'test/malignant/'+n;
  img=get_pytorch_image(img)
  temp=get_result_by_model(img, model_res)
  pred_label.append(temp)

from sklearn.metrics import accuracy_score,average_precision_score, f1_score,recall_score, precision_score
acc=accuracy_score(label1,pred_label)
f1=f1_score(label1,pred_label)
re=recall_score(label1,pred_label)
pre=precision_score(label1,pred_label)

print(acc)
print(f1)
print(re)
print(pre)

Performance of model 3

In [ ]:
pred_label=[]
for n,l in zip(name1,label1):
  if l==0:
    img=path+'test/benign/'+n;
  else:
    img=path+'test/malignant/'+n;
  img=get_pytorch_image(img)
  temp=get_result_by_model(img, model_dense)
  pred_label.append(temp)

from sklearn.metrics import accuracy_score,average_precision_score, f1_score,recall_score, precision_score
acc=accuracy_score(label1,pred_label)
f1=f1_score(label1,pred_label)
re=recall_score(label1,pred_label)
pre=precision_score(label1,pred_label)

print(acc)
print(f1)
print(re)
print(pre)

Performance of model 1

In [ ]:
pred_label=[]
for n,l in zip(name1,label1):
  if l==0:
    img=path+'test/benign/'+n;
  else:
    img=path+'test/malignant/'+n;
  img=get_pytorch_image(img)
  temp=get_result_by_model(img, model_inc)
  pred_label.append(temp)

from sklearn.metrics import accuracy_score,average_precision_score, f1_score,recall_score, precision_score
acc=accuracy_score(label1,pred_label)
f1=f1_score(label1,pred_label)
re=recall_score(label1,pred_label)
pre=precision_score(label1,pred_label)

print(acc)
print(f1)
print(re)
print(pre)

Performance of model 4

In [ ]:
pred_label=[]
for n,l in zip(name1,label1):
  if l==0:
    img=path+'test/benign/'+n;
  else:
    img=path+'test/malignant/'+n;
  img=get_pytorch_image(img)
  temp=get_result_by_model(img, model_next)
  pred_label.append(temp)

from sklearn.metrics import accuracy_score,average_precision_score, f1_score,recall_score, precision_score
acc=accuracy_score(label1,pred_label)
f1=f1_score(label1,pred_label)
re=recall_score(label1,pred_label)
pre=precision_score(label1,pred_label)

print(acc)
print(f1)
print(re)
print(pre)

Performance of model 5

In [ ]:
pred_label=[]
for n,l in zip(name1,label1):
  if l==0:
    img=path+'test/benign/'+n;
  else:
    img=path+'test/malignant/'+n;
  img=get_pytorch_image(img)
  temp=get_result_by_model(img, model_wres)
  pred_label.append(temp)

from sklearn.metrics import accuracy_score,average_precision_score, f1_score,recall_score, precision_score
acc=accuracy_score(label1,pred_label)
f1=f1_score(label1,pred_label)
re=recall_score(label1,pred_label)
pre=precision_score(label1,pred_label)

print(acc)
print(f1)
print(re)
print(pre)